In [21]:
import pandas as pd
from pathlib import Path
import os
import openpyxl
from collections import defaultdict


In [ ]:
repo_root = Path().cwd()
print(repo_root)

ref_path  = repo_root / "master_sheet_reference.xlsx"
data_path = repo_root / "assessment_guide.xlsx"


ref_sheets  = pd.read_excel(ref_path,  sheet_name=None)
data_sheets = pd.read_excel(data_path, sheet_name=None)

print("Reference file sheets:", list(ref_sheets.keys()))
print("Data file sheets     :", list(data_sheets.keys()))

c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash


c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Valid

Reference file sheets: ['P&M Schedule', 'P&M RIsk', 'Cost Estimation & Optimization', 'D&C - Quality', 'D&C - Procurement', 'D&C - HSE & Welfare', 'Innovation & Technology', 'Design & Technical', 'D&C - Construction', 'Strategy & Operations', 'Visuals Menu']
Data file sheets     : ['Main', 'Instructions', 'Glossary&Definitions', 'DocumentList', 'DevCoAssessmentInput(P&M)', 'DevCoAssessmentAnalysis(P&M)', 'InterviewQuestion(P&M)', 'AddDataPoint(P&M)', 'DevCoAssessmentInput(D&T)', 'DevCoAssessmentAnalysis(D&T)', 'InterviewQuestion(D&T)', 'AddDataPoint(D&T)', 'DevCoAssessmentInput(D&C)', 'DevCoAssessmentAnalysis(D&C)', 'InterviewQuestion(D&C)', 'AddDataPoint(D&C)', 'DevCoAssessmentInput(CE&O)', 'DevCoAssessmentAnalysis(CE&O)', 'InterviewQuestion(CE&O)', 'AddDataPoint(CE&O)', 'DevCoAssessmentInput(I&T)', 'DevCoAssessmentAnalysis(I&T)', 'InterviewQuestion(I&T)', 'AddDataPoint(I&T)', 'DevCoAssessmentInput(S&O)', 'DevCoAssessmentAnalysis(S&O)', 'InterviewQuestion(S&O)', 'AddDataPoint(S&O)', '

{'P&M Schedule':                           Table                               Column Visual
 0             AddDataPoint(P&M)                Additional Data Point   Card
 1             AddDataPoint(P&M)                                Value   Card
 2  DevCoAssessmentAnalysis(P&M)                  Assessment Criteria    NaN
 3  DevCoAssessmentAnalysis(P&M)                                Value    NaN
 4             AddDataPoint(P&M)                                 Year    NaN
 5             AddDataPoint(P&M)  Value Completed (Baseline) in M SAR    NaN
 6             AddDataPoint(P&M)    Value Completed (Actual) in M SAR    NaN
 7             AddDataPoint(P&M)  Value Completed (Forecast) in M SAR    NaN,
 'P&M RIsk':                            Table                 Column  Visual
 0              AddDataPoint(P&M)  Additional Data Point     NaN
 1              AddDataPoint(P&M)                  Value     NaN
 2   DevCoAssessmentAnalysis(P&M)    Assessment Criteria     NaN
 3   DevCoAssessme

In [25]:
ref_sheets['P&M Schedule']

,Table,Column,Visual
0,AddDataPoint(P&M),Additional Data Point,Card
1,AddDataPoint(P&M),Value,Card
2,DevCoAssessmentAnalysis(P&M),Assessment Criteria,NaN
3,DevCoAssessmentAnalysis(P&M),Value,NaN
4,AddDataPoint(P&M),Year,NaN
5,AddDataPoint(P&M),Value Completed (Baseline) in M SAR,NaN
6,AddDataPoint(P&M),Value Completed (Actual) in M SAR,NaN
7,AddDataPoint(P&M),Value Completed (Forecast) in M SAR,NaN


In [32]:
ref_tabs = []     

for sheet_name, df in ref_sheets.items():
    if {"Table", "Column"}.issubset(df.columns):
        table_cols = (
            df.groupby("Table")["Column"]
              .apply(list)
              .to_dict()
        )
        ref_tabs.append({ sheet_name: table_cols })
    else:
        print(f"Skipping sheet {sheet_name!r}: missing \"Table\" and \"Columm\" columns")

ref_tabs

Skipping sheet 'Visuals Menu': missing "Table" and "Columm" columns


[{'P&M Schedule': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Year',
    'Value Completed (Baseline) in M SAR',
    'Value Completed (Actual) in M SAR',
    'Value Completed (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}},
 {'P&M RIsk': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Risk Category',
    'Number of Risk Items',
    'Risk',
    'Source',
    'Potential Impact',
    'Risk Rating',
    'Risk Status',
    'Current Measures'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria',
    'Value',
    'Rating']}},
 {'Cost Estimation & Optimization': {'AddDataPoint(CE&O)': ['Additional Data Point',
    'Value',
    'Year',
    'Budget Value (Baseline) in M SAR',
    'Budget Value (Revised) in M SAR',
    'Budget Value (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(CE&O)': ['Assessment Criteria', 'Value', 'Rating'],
   'DevCoAssessmentInput(CE&O)': ['Data Point',
    'Input Value',
    'Da